In [1]:
#导入相关数据库
from KRData.HKData import HKFuture
import talib as ta
import pandas as pd
import numpy as np

In [2]:
#导入相关数据库
from KRData.HKData import HKFuture
import talib
import pandas as pd
import numpy as np

In [3]:
import datetime as dt

In [4]:
#连接数据库
hf = HKFuture('', '')

In [5]:
#取数据，由20110101-20190401之间HSI的数据
trade_dates=hf.get_main_contract_trade_dates('HSI','20110101','20190401')

In [39]:
ret_list=[]
ret_list2=[]
#设置一个循环来遍历每个合约的代码和对应的交易时间段
for code,tds in trade_dates.items():
    #输出结果
    print(f'code:{code}    {tds[0]}->{tds[-1]}')
    data=hf.get_bars(code,fields=['datetime','open','high','low','close','trade_date'],start=tds[0],end=tds[-1])
    data['last_close'] = data['close'].shift(1)
    #计算MA60
    data['ma60']=talib.MA(data['close'],timeperiod=60)
    data['gt_ma60']=data['close']>data['ma60']
    data['gt_ma60_1']=data['gt_ma60'].shift(1).fillna(0)
    #通过位运算确定金叉死叉情况，1为金叉，2为死叉
    data['ma60_type'] = (data['gt_ma60_1'].values << 1) + data['gt_ma60'].values
    #计算MACD
    data['macd'],data['diff'],data['dea']=ta.MACD(data.close.values,fastperiod=6,slowperiod=12,signalperiod=9)
    data['macd_gt0'] = (data['macd']>=0).astype(np.int8)
    data['macd_gt0_1'] = (data['macd_gt0'].shift(1).fillna(0)).astype(np.int8)
    #通过位运算计算红区和绿区
    data['macd_type'] = (data['macd_gt0_1'].values << 1) + data['macd_gt0'].values
    #取出MA60金叉死叉的两种情况
    ma60_type = data[(data['ma60_type']==1)|(data['ma60_type']==2)]
    #对应的时间做一个shift位移求用于之后求对应金叉死叉区间
    ma60_type['datetime_1'] = ma60_type['datetime'].shift(-1)
    #金叉情况
    d=ma60_type[ma60_type['ma60_type']==1].loc[:, ['datetime', 'datetime_1']]
    #这里增加了一个过滤条件，区间持续20分钟以上的才算一个
    for i, r in d[(d['datetime_1'] - d['datetime']) > dt.timedelta(minutes=20)].iterrows():
        ma60_df = data[r.datetime:r.datetime_1]
        macd_type = ma60_df[(ma60_df['macd_type']==1)|(ma60_df['macd_type']==2)]
        macd_type['macd_close'] = macd_type['close'].shift(-1)
        macd_type['macd_section_pnl'] = macd_type['macd_close'] - macd_type['close']
        #将金叉情况的区间pnl统计进来，然后把总的那个df赋值给ret
        ret = macd_type[macd_type['macd_type'] == 1].loc[:, ['macd_section_pnl','macd_close']]
        #用macd区间的收盘价作为入场价，ma60的收盘价作为出场价，统计不同macd区间入场对应的盈利
        ret['macd_close2ma60_close'] = ret['macd_close'] - ma60_df.iloc[-1]['close']
        ret['count'] = range(len(ret))
        ret_list.append(ret)

        
    #死叉情况
    d=ma60_type[ma60_type['ma60_type']==2].loc[:, ['datetime', 'datetime_1']]
    for i, r in d[(d['datetime_1'] - d['datetime']) > dt.timedelta(minutes=20)].iterrows():
        ma60_df = data[r.datetime:r.datetime_1]
        macd_type = ma60_df[(ma60_df['macd_type']==1)|(ma60_df['macd_type']==2)]
        macd_type['macd_close'] = macd_type['close'].shift(-1)
        macd_type['macd_section_pnl'] = macd_type['macd_close'] - macd_type['close']
        #将死叉情况的区间pnl统计进来，然后把总的那个df赋值给ret
        ret = macd_type[macd_type['macd_type'] == 2].loc[:, ['macd_section_pnl','macd_close']]
        #用macd区间的收盘价作为入场价，ma60的收盘价作为出场价，统计不同macd区间入场对应的盈利
        ret['macd_close2ma60_close'] = ma60_df.iloc[-1]['close']-ret['macd_close']
        ret['count'] = range(len(ret))
        ret_list2.append(ret)
    

code:HSI1101    2011-01-03 00:00:00->2011-01-28 00:00:00


D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

code:HSI1102    2011-01-31 00:00:00->2011-02-25 00:00:00
code:HSI1103    2011-02-28 00:00:00->2011-03-30 00:00:00
code:HSI1104    2011-03-31 00:00:00->2011-04-28 00:00:00
code:HSI1105    2011-04-29 00:00:00->2011-05-30 00:00:00
code:HSI1106    2011-05-31 00:00:00->2011-06-29 00:00:00
code:HSI1107    2011-06-30 00:00:00->2011-07-28 00:00:00
code:HSI1108    2011-07-29 00:00:00->2011-08-30 00:00:00
code:HSI1109    2011-08-31 00:00:00->2011-09-30 00:00:00
code:HSI1110    2011-10-03 00:00:00->2011-10-28 00:00:00
code:HSI1111    2011-10-31 00:00:00->2011-11-29 00:00:00
code:HSI1112    2011-11-30 00:00:00->2011-12-29 00:00:00
code:HSI1201    2011-12-30 00:00:00->2012-01-30 00:00:00
code:HSI1202    2012-01-31 00:00:00->2012-02-28 00:00:00
code:HSI1203    2012-02-29 00:00:00->2012-03-29 00:00:00
code:HSI1204    2012-03-30 00:00:00->2012-04-27 00:00:00
code:HSI1205    2012-04-30 00:00:00->2012-05-30 00:00:00
code:HSI1206    2012-05-31 00:00:00->2012-06-28 00:00:00
code:HSI1207    2012-06-29 00:0

In [40]:
total_ret1 = pd.concat([ ret for ret in ret_list])

In [41]:
total_ret2 = pd.concat([ ret for ret in ret_list2])

In [42]:
total_ret1.head()

,macd_section_pnl,macd_close,macd_close2ma60_close,count
2011-01-03 11:24:00,-28.0,23388.0,-12.0,0.0
2011-01-03 11:45:00,0.0,23402.0,2.0,1.0
2011-01-04 10:10:00,72.0,23524.0,8.0,0.0
2011-01-04 10:38:00,-12.0,23527.0,11.0,1.0
2011-01-04 10:51:00,-12.0,23513.0,-3.0,2.0


In [ ]:
#金叉统计情况

In [84]:
print(total_ret1.groupby('count')['macd_section_pnl'].describe())

        count        mean        std    min     25%    50%     75%     max
count                                                                     
0.0    7669.0   14.152693  50.095082 -337.0   -7.00    2.0   23.00  1018.0
1.0    4250.0    0.457882  36.092325 -451.0  -12.00   -5.0    2.00   366.0
2.0    2106.0    0.783476  37.011453 -320.0  -12.00   -5.0    1.00   513.0
3.0     938.0    2.320896  45.134856 -307.0  -11.00   -5.0    2.00   559.0
4.0     382.0    0.633508  31.579533 -135.0  -12.00   -5.0    1.00   239.0
5.0     149.0   -1.181208  27.461780 -148.0  -10.00   -5.0    1.00   127.0
6.0      61.0   -0.163934  30.485068  -60.0  -10.00   -5.0    3.00   130.0
7.0      27.0    6.444444  31.653574  -20.0  -11.00   -6.0    9.50   111.0
8.0      14.0   -3.642857  15.494948  -21.0  -10.25   -5.5   -2.25    45.0
9.0       4.0  -11.250000   6.652067  -18.0  -15.75  -12.0   -7.50    -3.0
10.0      1.0  121.000000        NaN  121.0  121.00  121.0  121.00   121.0
11.0      1.0  -12.000000

In [45]:
#按MACD红区进行统计
grouped = total_ret1.groupby('count')

In [85]:
df1=grouped.apply(calc)

D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in long_scalars
  after removing the cwd from sys.path.
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [88]:
df1

,pos_count,neg_count,winloss,winper,pos_sum,neg_sum,total_sum,avg_profit,avg_win,avg_loss
count,,,,,,,,,,
0.0,4720.0,2949.0,1.600543,0.615465,78055.0,-92501.0,-14446.0,-1.883688,16.537076,-31.366904
1.0,2663.0,1587.0,1.678009,0.626588,38061.0,-42140.0,-4079.0,-0.959765,14.292527,-26.553245
2.0,1282.0,824.0,1.555825,0.608737,17929.0,-18554.0,-625.0,-0.296771,13.985179,-22.516990
3.0,555.0,383.0,1.449086,0.591684,7412.0,-6592.0,820.0,0.874200,13.354955,-17.211488
4.0,248.0,134.0,1.850746,0.649215,3000.0,-2841.0,159.0,0.416230,12.096774,-21.201493
5.0,82.0,67.0,1.223881,0.550336,937.0,-1296.0,-359.0,-2.409396,11.426829,-19.343284
6.0,36.0,25.0,1.440000,0.590164,581.0,-672.0,-91.0,-1.491803,16.138889,-26.880000
7.0,19.0,8.0,2.375000,0.703704,213.0,-262.0,-49.0,-1.814815,11.210526,-32.750000
8.0,12.0,2.0,6.000000,0.857143,132.0,-171.0,-39.0,-2.785714,11.000000,-85.500000


In [ ]:
#winloss为盈亏比，winper是盈亏百分比，pos_sum是总盈利,neg_sum是总亏损，total_sum是整体盈利
#avg_profit是总平均盈亏，avg_win是平均盈亏,avg_loss是平均损失

In [81]:
def calc(df):
    pos_count = df[df['macd_close2ma60_close']>0]['macd_close2ma60_close'].count()
    neg_count = df[df['macd_close2ma60_close']<=0]['macd_close2ma60_close'].count()
    winloss=pos_count/neg_count
    winper=pos_count/(pos_count+neg_count)
    pos_sum=df[df['macd_close2ma60_close']>0]['macd_close2ma60_close'].sum()
    neg_sum=df[df['macd_close2ma60_close']<=0]['macd_close2ma60_close'].sum()
    total_sum=pos_sum+neg_sum
    avg_profit=total_sum/(pos_count+neg_count)
    avg_win=pos_sum/pos_count
    avg_loss=neg_sum/neg_count

    return pd.Series({'pos_count': pos_count, 'neg_count':neg_count, 'winloss':winloss,'winper':winper,'pos_sum':pos_sum,
                     'neg_sum':neg_sum,'total_sum':total_sum,'avg_profit':avg_profit,'avg_win':avg_win,'avg_loss':avg_loss})

In [ ]:
#死叉统计情况

In [89]:
print(total_ret2.groupby('count')['macd_section_pnl'].describe())

        count       mean        std    min    25%   50%    75%    max
count                                                                
0.0    7289.0 -15.105090  57.010653 -775.0 -22.00  -2.0   8.00  594.0
1.0    3966.0   0.123550  39.515494 -501.0  -1.00   6.0  13.00  453.0
2.0    1823.0   0.744377  38.826480 -465.0  -1.00   6.0  13.00  521.0
3.0     765.0  -0.881046  31.945064 -215.0  -1.00   6.0  13.00   68.0
4.0     297.0  -1.629630  35.863257 -322.0  -2.00   6.0  13.00  149.0
5.0     121.0  -4.528926  37.895267 -236.0  -5.00   6.0  11.00   54.0
6.0      55.0  -2.436364  38.421523 -228.0 -13.00   6.0  15.50   36.0
7.0      20.0   0.800000  47.672788 -131.0  -2.25   6.0  11.00  116.0
8.0       4.0  11.500000  12.124356   -3.0   3.75  13.5  21.25   22.0
9.0       2.0 -36.500000  16.263456  -48.0 -42.25 -36.5 -30.75  -25.0
10.0      1.0 -29.000000        NaN  -29.0 -29.00 -29.0 -29.00  -29.0
11.0      1.0 -13.000000        NaN  -13.0 -13.00 -13.0 -13.00  -13.0


In [90]:
#按MACD绿区进行统计
grouped2 = total_ret2.groupby('count')

In [97]:
df2=grouped2.apply(calc)
df2

D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in long_scalars
  after removing the cwd from sys.path.
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


,pos_count,neg_count,winloss,winper,pos_sum,neg_sum,total_sum,avg_profit,avg_win,avg_loss
count,,,,,,,,,,
0.0,4548.0,2741.0,1.659248,0.623954,75966.0,-84023.0,-8057.0,-1.105364,16.703166,-30.654141
1.0,2512.0,1454.0,1.727648,0.633384,37505.0,-36395.0,1110.0,0.279879,14.930334,-25.030949
2.0,1170.0,653.0,1.791730,0.641799,15875.0,-16615.0,-740.0,-0.405924,13.568376,-25.444104
3.0,501.0,264.0,1.897727,0.654902,6402.0,-7166.0,-764.0,-0.998693,12.778443,-27.143939
4.0,190.0,107.0,1.775701,0.639731,2964.0,-3240.0,-276.0,-0.929293,15.600000,-30.280374
5.0,91.0,30.0,3.033333,0.752066,1524.0,-1614.0,-90.0,-0.743802,16.747253,-53.800000
6.0,36.0,19.0,1.894737,0.654545,758.0,-377.0,381.0,6.927273,21.055556,-19.842105
7.0,11.0,9.0,1.222222,0.550000,355.0,-145.0,210.0,10.500000,32.272727,-16.111111
8.0,1.0,3.0,0.333333,0.250000,26.0,-126.0,-100.0,-25.000000,26.000000,-42.000000


In [ ]:
#winloss为盈亏比，winper是盈亏百分比，pos_sum是总盈利,neg_sum是总亏损，total_sum是整体盈利
#avg_profit是总平均盈亏，avg_win是平均盈亏,avg_loss是平均损失

In [96]:
def calc(df):
    pos_count = df[df['macd_close2ma60_close']>0]['macd_close2ma60_close'].count()
    neg_count = df[df['macd_close2ma60_close']<=0]['macd_close2ma60_close'].count()
    winloss=pos_count/neg_count
    winper=pos_count/(pos_count+neg_count)
    pos_sum=df[df['macd_close2ma60_close']>0]['macd_close2ma60_close'].sum()
    neg_sum=df[df['macd_close2ma60_close']<=0]['macd_close2ma60_close'].sum()
    total_sum=pos_sum+neg_sum
    avg_profit=total_sum/(pos_count+neg_count)
    avg_win=pos_sum/pos_count
    avg_loss=neg_sum/neg_count

    return pd.Series({'pos_count': pos_count, 'neg_count':neg_count, 'winloss':winloss,'winper':winper,'pos_sum':pos_sum,
                     'neg_sum':neg_sum,'total_sum':total_sum,'avg_profit':avg_profit,'avg_win':avg_win,'avg_loss':avg_loss})

In [ ]:
#输出统计结果

In [23]:
result1=total_ret1.groupby('count')['macd_section_pnl'].describe()

In [25]:
result1.to_excel('金叉统计描述.xlsx')

In [99]:
df1.to_excel('MACD红区空单策略结果.xlsx')

In [34]:
result3=total_ret2.groupby('count')['macd_section_pnl'].describe()

In [35]:
result3.to_excel('死叉统计描述.xlsx')

In [100]:
df2.to_excel('MACD绿区多单策略结果.xlsx')